<a href="https://colab.research.google.com/github/tushar3572/Gout-disease-prediction/blob/main/Goat_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install xgboost==1.7.6


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 6.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report
import joblib

In [3]:
from google.colab import files
uploaded = files.upload()   # Upload your dataset_gout_balanced.csv
df = pd.read_csv("dataset_gout_balanced.csv")

print("✅ Data loaded. Shape:", df.shape)
print(df.head())

Saving dataset_gout_balanced.csv to dataset_gout_balanced.csv
✅ Data loaded. Shape: (10316, 154)
   SDDSRVYR  RIDSTATR  RIAGENDR  RIDAGEYR  RIDRETH1  RIDRETH3  RIDEXMON  \
0      10.0       2.0       2.0      66.0       4.0       4.0       2.0   
1      10.0       2.0       2.0      66.0       5.0       6.0       2.0   
2      10.0       2.0       2.0      75.0       4.0       4.0       1.0   
3      10.0       2.0       1.0      56.0       5.0       6.0       2.0   
4      10.0       2.0       1.0      67.0       3.0       3.0       1.0   

   DMQMILIZ   DMQADFC  DMDBORN4  ...  LBDSCASI  LBXSCH  LBDSCHSI  LBXSTP  \
0       2.0  1.486641       1.0  ...     2.300   157.0     4.060     7.3   
1       2.0  1.486641       2.0  ...     2.375   210.0     5.431     7.1   
2       2.0  1.486641       1.0  ...     2.475   180.0     4.655     7.0   
3       2.0  1.486641       2.0  ...     2.350   239.0     6.181     7.1   
4       1.0  1.000000       1.0  ...     2.350   188.0     4.862     6.4

In [4]:
TARGET = "gout"   # column containing labels (0 = no gout, 1 = gout)
X = df.drop(TARGET, axis=1)   # all features
y = df[TARGET]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

In [7]:
model = RandomForestClassifier(
    n_estimators=300,
    random_state=42,
    class_weight="balanced"
)
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=300,
                       random_state=42)

In [8]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:,1]

print("\n📊 Model Performance:")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


📊 Model Performance:
Accuracy: 1.0
F1 Score: 1.0
ROC-AUC: 1.0

Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1032
         1.0       1.00      1.00      1.00      1032

    accuracy                           1.00      2064
   macro avg       1.00      1.00      1.00      2064
weighted avg       1.00      1.00      1.00      2064



In [9]:
joblib.dump(model, "gout_prediction_model.joblib")
print("\n💾 Model saved as gout_prediction_model.joblib")


💾 Model saved as gout_prediction_model.joblib


In [10]:
new_patient = X_train.median().to_frame().T

prediction = model.predict(new_patient)[0]
probability = model.predict_proba(new_patient)[0,1]

print("\n🧍 New Patient Prediction:")
print("Predicted Class:", "Gout" if prediction==1 else "No Gout")
print("Probability of Gout:", round(probability, 3))


🧍 New Patient Prediction:
Predicted Class: Gout
Probability of Gout: 0.587


In [11]:
all_preds = model.predict(X)
all_probs = model.predict_proba(X)[:, 1]

# Add results back to your dataframe
df["Predicted_Class"] = all_preds
df["Probability_of_Gout"] = all_probs

# Save predictions to CSV
df.to_csv("gout_predictions.csv", index=False)
print("✅ Predictions for the full dataset saved as gout_predictions.csv")

✅ Predictions for the full dataset saved as gout_predictions.csv
